Understanding human emotions paves the way to understanding people's needs better and, ultimately, providing better service. Speech Emotion Recognition (SER) is one of these tasks, aiming to recognize the emotion from speech irrespective of the semantic contents. </br></br>

<b>Data Description:</b> This dataset is a subset of the Toronto emotional speech set (TESS) in which a set of 200 target words were spoken in the carrier phrase "Say the word _____' by multiple actresses (aged 26 and 64 years), and recordings were made of the set portraying each of seven emotions. In this assignment the dataset used in this assignment contains 400 audio clips for 4 emotions: sad, angry, happy, and fear. </br></br>

Please download the dataset here: https://drive.google.com/drive/u/2/folders/14GKIR6RYKCYd-L3ppqfoHMH0anTgiz8H

<h1>Step 1: Split the dataset into training and testing sets.</h1></br>
In the dataset, we have 100 audio clips for each of the abovementioned 4 emotions. For each emotion category, randomly select 70 as training samples and the remaining 30 as testing samples. 

<h1>Step 2: Exploratory Data Analysis.</h1></br>
This stage is the very initial stage of your data analysis. You may want to know the label distribution of the dataset. You may also want to select sample audio, listen to them and plot them in both the time and frequency domain.

<h1>Step 3: Acoustic Feature Extraction.</h1></br>
You will extract acoustic features by using librosa or openSMILE. They are both popular Python libraries for automatic acoustic feature extraction.</br>
librosa: https://librosa.org/doc/main/index.html</br>
openSMILE: https://audeering.github.io/opensmile-python/

<h1>Step 4: Feature Post-processing.</h1></br>
Please refer to the slide of this class. </br>
Link: https://docs.google.com/presentation/d/1GUF_CaEy5rSnqnlOkxYhHGhfp6-1epCE5Q5BsYOfqt8/edit#slide=id.g1c126e18fd0_2_36

<h1>Step 5: Build your audio emotion recognition model.</h1></br>
Provide the extracted set of acoustic features from the training dataset to your classification model. Note that this is a multi-class classification problem. Please select appropriate classifiers since some of the classifiers are not applicable here (e.g., logistic regression).

<h1>Step 6: Model evaluation.</h1></br>
Evaluate your model performance with the testing dataset. Please compare the performance of different classifiers using the same acoustic feature and the performance of the same classifier using different acoustic features. Finally, discuss your experimental results and submit the assignment report.